In [ ]:
%matplotlib inline
from helpers import *

# import 'Sequential' is a linear stack of neural network layers. Will be used to build the feed-forward CNN
from keras.models import Sequential 
# import the "core" layers from Keras (these are the most common layers)
from keras.layers import Dense, Dropout, Activation, Flatten
# import the convolutional layers that will help us efficiently train on image data
from keras.layers import Conv2D, MaxPooling2D
# these utilities will help us transform our data
from keras.utils import np_utils

%load_ext autoreload
%autoreload 2

#### Overview
**Convolutional neural networks** (CNNs) are used to classify images. Owever our task does not consist in classifying the whole image but only its patches of 16x16 pixels. Therefore, we will need to properly choose the input of our CNN, i.e. it will have (at least) one input per patch so to classify each patch either as **road** or as **backgrund**. Moreover, each input shouldn't be just a patch of 16x16 pixels but should be a bigger "window" so to give the model a context for each patch. **The size of this window has to be refined**. A minor problem is to define the context for the patches in the border. This problem can be tackled by extending each image by mirroring its border so that also the border patches have a meaningful context. 


**TODO** try with more inputs, e.g. one input per 8x8 patch.

### - Load data

In [ ]:
# Loaded a set of images
n = 20

imgs, gt_imgs = load_images(n)

imgs[0].shape, gt_imgs[0].shape

### - Define the window width

In [ ]:
window_width = 72

### - Extend images by mirroring
First, as explained, we extend the input images.

In [ ]:
imgs_ext = extend_images(imgs, window_width)
imgs_ext.shape

In [ ]:
# just show an example
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(20, 10)
axs[0].imshow(imgs[1], cmap='Greys_r', vmin=0, vmax=1)

mirror_width = int((window_width-16)/2)
img_ext = imgs_ext[1]
img_ext[mirror_width:-mirror_width, mirror_width-1:mirror_width+1] = np.array((0, 1, 0))
img_ext[mirror_width:-mirror_width, -mirror_width-1:-mirror_width+1] = np.array((0, 1, 0))
img_ext[mirror_width-1:mirror_width+1, mirror_width:-mirror_width] = np.array((0, 1, 0))
img_ext[-mirror_width-1:-mirror_width+1, mirror_width:-mirror_width] = np.array((0, 1, 0))
axs[1].imshow(img_ext, cmap='Greys_r', vmin=0, vmax=1)

plt.tight_layout()

### - For now avoid cross validation, just split the datasest in test and train. 

In [ ]:
train_ratio = 0.8
train, test = split_train_test(imgs_ext, gt_imgs, train_ratio=train_ratio, seed=1)
train.imgs.shape, train.gt_imgs.shape, test.imgs.shape, test.gt_imgs.shape 

### - Generate the inpus and the outputs
Now we have to transform our images into **inputs** and **outputs** that will be fed to the CNN. From each image we extract **one input per patch** that had to be classified to **road (1)** or **background (0)** (400x400/16x16=625 inputs). Lastly, we will reshape each input to fulfill the requirements of the tensorflow library. 

In [ ]:
train.X, train.Y = imgs_to_inputs_outputs(train.imgs, train.gt_imgs, window_width)
test.X, test.Y = imgs_to_inputs_outputs(test.imgs, test.gt_imgs, window_width)
train.X.shape, train.Y.shape, test.X.shape, test.Y.shape

In [ ]:
# verify that the windows have the correct label
nrow, ncol = 4, 4
fig, axs = plt.subplots(nrow, ncol)
fig.set_size_inches(5*nrow, 5*ncol)

ind = int((window_width-16)/2)
for k, window in enumerate(np.random.choice(range(train.X.shape[0]), size = nrow*ncol)):
    i = int(k / ncol)
    j = int(k % ncol)
    
    # just show the patch in the middle
    wind = train.X[window]
    
    wind[ind:ind+16, ind-1:ind] = np.array([0, 1, 0])
    wind[ind:ind+16, -ind:-ind+1] = np.array([0, 1, 0])
    wind[ind-1:ind, ind:ind+16] = np.array([0, 1, 0])
    wind[-ind:-ind+1, ind:ind+16] = np.array([0, 1, 0])

    axs[i, j].imshow(wind, cmap='Greys_r', vmin=0, vmax=1)
    axs[i, j].set_title("road" if (train.Y[window][1]==1) else "background")

plt.tight_layout()

Recap on **train** (and test) object:
- **train.imgs**: (expanded) images of the training set. **shape**=(#train images, 456, 456, 3)
- **train.gt_imgs**: respective groundtruth images. **shape**=(#train images, 400, 400)
- **train.X**: inputs extracted from train.imgs. **shape**=(#total windows, 72, 72, 3)
- **train.Y**: outputs extracted from train.gt_imgs. **shape**=(#total windows, 2)

### - Build the CNN model

In [ ]:
nclasses = 2
model = Sequential()
# 32 convolution filters
# 3 rows in convolution kernel
# 3 columns in convolution kernel
# (1,28,28) is the shape of one input
# strides=(1, 1) by default (step size?)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=train.X[0].shape))
model.add(Conv2D(32, (3, 3), activation='relu'))

# MaxPooling2D is a way to reduce the number of parameters in our model by sliding 
# a 2x2 pooling filter across the previous layer and taking the max of the 4 values 
# in the 2x2 filter.
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # a method for regularizing our model in order to prevent overfitting.

# a convolutional neural network always ends with a fully connected layer followe by the ouput
# layer
# first flatten the weights of the convolution
model.add(Flatten())
# 128 = output size of the dense layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# 2 = output size of the output dense layer (we have 2 classes)
model.add(Dense(nclasses, activation='softmax'))

model.output_shape

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### - Fit the model on the train data

In [ ]:
model.fit(train.X, train.Y, batch_size=32, epochs=1, verbose=1)

### - Evaluate the model on the test data

In [ ]:
# score = model.evaluate(test.X, test.Y, verbose=1)
# score

In [ ]:
Z = model.predict_classes(test.X, verbose=1)
stats(Z, test.Y[:, 1])

In [ ]:
i = 0
display_ith_prediction(test, Z, i, window_width)

**problem**: it predicts always the same class [check here](https://stackoverflow.com/questions/41488279/neural-network-always-predicts-the-same-class)